# MLOps with TensorFlow (using TensorFlow-Extended)

MLOps has appeared as a popular workflow for machine learning. As more and more industries are adopting machine learning, there is an enormous need for standardizing and streamlining steps like,
* data ingestion
* feature generation
* model training
* evaluation
* etc.

MLOps focuses on providing a standard workflow (a series of steps) that perform these tasks. Once a model is trained and evaluated to be a good quality model, it needs to be served. Productionization is the process of deploying the model on some infrastructure (e.g. a single machine or a cluster) and making it accessible for predictions via an web based API (e.g. REST API).

---
**Note**: We are using a slightly older version of `tfx` (`==1.6.0`) and `tensorflow`(`==2.7.3`) here. The `InteractiveContext` we use to run components of `tfx` we use here is broken in the latest versions `> 1.6.0`. If you try to run the same component multiple times in the same context, in a later version, you may get an error starts like

```
tensorflow.python.framework.errors_impl.AlreadyExistsError: Given node already exists: type_id: ...
```

This is likely a bug due to `tfx` not being able to differentiate between executions of the same component run at different timestamps. 

---

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch15-TFX-for-MLOps-in-TF2/tfx/15.1_MLOps_with_tensorflow.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>



# Importing necessary libraries

In [4]:
import tensorflow as tf
import tfx
import apache_beam

import requests
import requests
import os
import time
import pandas as pd

import random
import shutil
import tempfile
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import CSVLogger
import numpy as np
import pickle
from tensorflow.keras.models import load_model, Model
import glob
from functools import partial
import absl

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

print("TensorFlow version: {}".format(tf.__version__))
print("TFX version: {}".format(tfx.__version__))
print("Apache beam version: {}".format(apache_beam.__version__))

TensorFlow version: 2.7.3
TFX version: 1.6.0
Apache beam version: 2.39.0


2022-07-30 15:05:30.396807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:05:30.408610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:05:30.408968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [5]:
# Remove the pipeline folder and other generated artefacts if they exist
if os.path.exists('pipeline'):
    shutil.rmtree('pipeline')
if os.path.exists('forestfires-model-pushed'):
    shutil.rmtree('forestfires-model-pushed')
    
if os.path.exists('forest_fires_constants.py'):
    os.remove('forest_fires_constants.py')
if os.path.exists('forest_fires_transform.py'):
    os.remove('forest_fires_transform.py')
if os.path.exists('forest_fires_trainer.py'):
    os.remove('forest_fires_trainer.py')
if os.path.exists('forest_fires_trainer_v2.py'):
    os.remove('forest_fires_trainer_v2.py')

## Download the data

We are going to use a dataset contining information about forest fires in the USA. The dataset is downloaded from [this link](http://archive.ics.uci.edu/ml/datasets/Forest+Fires). The dataset contains the following features.

* X - x-axis spatial coordinate within the Montesinho park map
* Y - y-axis spatial coordinate within the Montesinho park map
* month - month of the year
* day - day of the week
* FFMC - Fine Fuel Moisture Code (FFMC) represents fuel moisture of forest litter fuels under the shade of a forest canopy
* DMC - DMC is a numerical rating of the average moisture content of soil
* DC - The Drought Code (DC) represents drying deep into the soil.
* ISI - ISI index from the FWI system (The Initial Spread Index (ISI) is an expected rate of fire spread)
* temp - temperature in Celsius degrees
* RH - relative humidity in %
* wind - wind speed in km/h
* rain - outside rain in mm/m2
* area - the burned area of the forest (in ha)

In [6]:
# Section 15.1

# Code listing 15.1

# Downloading the data

import os
import requests
import tarfile

import shutil

# Retrieve the data
if not os.path.exists(os.path.join('data', 'csv', 'forestfires.csv')):
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists(os.path.join('data', 'csv')):
        os.makedirs(os.path.join('data', 'csv'))
    
    # Write to a file
    with open(os.path.join('data', 'csv', 'forestfires.csv'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.csv file already exists.")
    
    
if not os.path.exists(os.path.join('data', 'forestfires.names')):
    
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Write to a file
    with open(os.path.join('data', 'forestfires.names'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.names file already exists.")



The forestfires.csv file already exists.
The forestfires.names file already exists.


## Separate train/valid and test datasets

In [7]:
import pandas as pd

df = pd.read_csv(os.path.join('data', 'csv', 'forestfires.csv'), index_col=None, header=0)
train_df = df.sample(frac=0.95, random_state=random_seed)
test_df = df.loc[~df.index.isin(train_df.index), :]

train_path = os.path.join('data','csv','train')
os.makedirs(train_path, exist_ok=True)
test_path = os.path.join('data','csv','test')
os.makedirs(test_path, exist_ok=True)

train_df.to_csv(os.path.join(train_path, 'forestfires.csv'), index=False, header=True)
test_df.to_csv(os.path.join(test_path, 'forestfires.csv'), index=False, header=True)

# Defining the pipeline root

In [8]:
# Section 15.1

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_pipeline_root = os.path.join(os.getcwd(), 'pipeline', 'examples', 'forest_fires_pipeline')
print(_pipeline_root)

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline


# Defining an interactive context

Context runs various steps in the pipeline and maintains state between different steps. Interactive means that, it can be called/run via multiple calls, instead of doing via a single call. This is suited when running things on a notebook, where code is quite isolated.

In [9]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.

context = InteractiveContext(pipeline_name = "forest_fires", pipeline_root=_pipeline_root)

INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'


# Define an Example Generator

This example generator generates examples (as `TFRecord` objects) from a CSV file. 

In [10]:
# Section 15.1
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

# You can change the configuration of generating train-eval split by passing a SplitConfig as below
# Note that we have not used it in the CsvExampleGen
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

# Define the example generator
example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv', 'train'))
# Run the example generator
context.run(example_gen, enable_cache=False)

INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data data/csv/train/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:26223,xor_checksum:1659157531,sum_checksum:1659157531"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Some results produced by `CsvExampleGen`

In [11]:
# Print outputs
print(example_gen.outputs)

# Accessing various variables in the example generator
artifact = example_gen.outputs['examples'].get()[0]

# Print some of that
print("Artifact split names: {}".format(artifact.split_names))
print("Artifact URI: {}".format(artifact.uri))


{'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 14
uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:26223,xor_checksum:1659157531,sum_checksum:1659157531"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.6.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties 

## Print a few examples generated from the CSV file

In [12]:
# Section 15.1
# Code listing 15.2

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 2 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)


2022-07-30 15:05:34.229638: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-30 15:05:34.229897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:05:34.230267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 15:05:34.230573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

features {
  feature {
    key: "DC"
    value {
      float_list {
        value: 605.7999877929688
      }
    }
  }
  feature {
    key: "DMC"
    value {
      float_list {
        value: 141.3000030517578
      }
    }
  }
  feature {
    key: "FFMC"
    value {
      float_list {
        value: 95.0999984741211
      }
    }
  }
  feature {
    key: "ISI"
    value {
      float_list {
        value: 17.700000762939453
      }
    }
  }
  feature {
    key: "RH"
    value {
      int64_list {
        value: 43
      }
    }
  }
  feature {
    key: "X"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "Y"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "area"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "day"
    value {
      bytes_list {
        value: "tue"
      }
    }
  }
  feature {
    key: "month"
    value {
      bytes_list {
        value: "aug"
    

# Generate statistics of the data

Statistics compute various metrics. For, 
* continuous variables
  * like mean, standard deviation, count of zeros, median, min/max, etc.  
* categorical variables
  * count, missing count, number of unique values, etc.

In [13]:
# Section 15.1

from tfx.components import StatisticsGen

# Define the statistic generator
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
# Run the statistic  generator
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## View the statistics of train / eval data

In [14]:
context.show(statistics_gen.outputs['statistics'])

# Infer the schema of the data

The schema is like a blueprint for how the data should look like. It define things like data types of the data, minimum/maximum values allowed, etc.

In [15]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Print `schema_gen` artifact location

In [16]:
print(schema_gen.outputs['schema']._artifacts[0].uri)

/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3


## Display the inferred schema

Schema can be displayed as dataframes. TFX uses the `tensorflow_data_validation` library to do that. To do this manually, do the following.

```
schema = tfdv.load_schema_text(
    os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt')
) 
features_df, domain_df = tfdv.utils.display_util.get_schema_dataframe(schema)
tfdv.utils.display_util.display(domain_df)
```

In [17]:
import tensorflow_data_validation as tfdv

context.show(schema_gen.outputs['schema'])


Type  Presence Valency   Domain
Feature name                                   
'DC'           FLOAT  required  single        -
'DMC'          FLOAT  required  single        -
'FFMC'         FLOAT  required  single        -
'ISI'          FLOAT  required  single        -
'RH'             INT  required  single        -
'X'              INT  required  single        -
'Y'              INT  required  single        -
'area'         FLOAT  required  single        -
'day'         STRING  required  single    'day'
'month'       STRING  required  single  'month'
'rain'         FLOAT  required  single        -
'temp'         FLOAT  required  single        -
'wind'         FLOAT  required  single        -

Values
Domain                                                                                     
'day'                                       'fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed'
'month'  'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'oct', 'sep', 'nov'

https://www.tensorflow.org/tfx/tutorials/tfx/components

# Transforming data into features with `tensorflow_transform`

In this step, we transform the columnar data we have to features. For example, values in some columns will be normalized (e.g. z-score normalization), one-hot encoded, etc.

## Defining feature types

Below we define some constants that will dictate how each column is going to be treated.

In [18]:
%%writefile forest_fires_constants.py

# Section 15.1
# Code listing 15.3

# Vocab based features

VOCAB_FEATURE_KEYS = ['day','month']

# Vocab based features are in fact categorical features
# Categorical features are assumed to each have a maximum value in the dataset
MAX_CATEGORICAL_FEATURE_VALUES = [7, 12]

# Dense features (these will go to the model as they are)

DENSE_FLOAT_FEATURE_KEYS = ['DC', 'DMC', 'FFMC', 'ISI', 'rain', 'temp', 'wind', 'X', 'Y']

# Bucketized features 

# Number of buckets used by tf.transform for encoding each feature.
BUCKET_FEATURE_BOUNDARIES = [(33, 66)]
BUCKET_FEATURE_KEYS = ['RH']

# Label features

LABEL_KEY = 'area'


def transformed_name(key):
    """ Define a function that will add a suffix to the feature name """
    
    return key + '_xf'


Writing forest_fires_constants.py


## Processing the columns to features

Here we define a function `preprocessing_fn()` that instructs how each column should be treated. To do the computations, we use the `tensorflow_transform` library.

In [19]:
%%writefile forest_fires_transform.py

# Section 15.1
# Code listing 15.4

import tensorflow as tf
import tensorflow_transform as tft

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
    
  # Treating dense features
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        sparse_to_dense(inputs[key])
    )

  # Treating categorical features  
  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        sparse_to_dense(inputs[key]),
        num_oov_buckets=1)

  # Treating bucketized features
  for key, boundary in zip(_BUCKET_FEATURE_KEYS, _BUCKET_FEATURE_BOUNDARIES):
    outputs[_transformed_name(key)] = tft.apply_buckets(
        sparse_to_dense(inputs[key]), bucket_boundaries=[boundary])

  # Treating the label feature
  outputs[_transformed_name(_LABEL_KEY)] = sparse_to_dense(inputs[_LABEL_KEY])

  return outputs

def sparse_to_dense(x):
    
    return tf.squeeze(
        tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1])),
        axis=1
    )


Writing forest_fires_transform.py


## Running the transform step

With the preprocessing logic defined, we can run the Transform component of the TFX pipeline.

In [20]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'),
    disable_analyzer_cache=True,
)

context.run(transform)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/forest_fires_transform.py' (including modules: ['forest_fires_constants', 'forest_fires_transform']).
INFO:absl:User module package has hash fingerprint version e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/bin/python', '/tmp/tmpksdc8gri/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp34g1958y', '--dist-dir', '/tmp/tmpxjihijcv']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying forest_fires_constants.py -> build/lib
copying forest_fires_transform.py -> build/lib
installing to /tmp/tmp34g1958y
running install
running install_lib
copying build/lib/forest_fires_constants.py -> /tmp/tmp34g1958y
copying build/lib/forest_fires_transform.py -> /tmp/tmp34g1958y
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt


/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Tra

writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp34g1958y/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3.8.egg-info
running install_scripts
creating /tmp/tmp34g1958y/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7.dist-info/WHEEL
creating '/tmp/tmpxjihijcv/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3-none-any.whl' and adding '/tmp/tmp34g1958y' to it
adding 'forest_fires_constants.py'
adding 'forest_fires_transform.py'
adding 'tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7.dist-info/METADATA'
adding 'tfx_user_code_Tran

INFO:absl:Successfully installed '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+e88186202a668eaf146fd9420d590396b3c2ecf93893e4e680f2d6fe3c1b78f7-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/bin

...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"eval\", \"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 6
        type_id: 18
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/pre_transform_schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_stats: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 7
        type_id: 16
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/pre_transform_stats/4"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_stats"
          }
        }
        custom_p

## Print a few examples generated from the transformer

In [21]:
# Section 15.1
# Code listing 15.5

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_LABEL_KEY = forest_fires_constants.LABEL_KEY

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.

example_records = []
float_features = [f+'_xf' for f in _DENSE_FLOAT_FEATURE_KEYS + ['area']]
int_features = [f+'_xf' for f in _BUCKET_FEATURE_KEYS + _VOCAB_FEATURE_KEYS]
for tfrecord in dataset.take(5):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  record = [example.features.feature[f].int64_list.value for f in int_features] + [example.features.feature[f].float_list.value for f in float_features]
  example_records.append(record)
  print(example)
  print("="*50)

features {
  feature {
    key: "DC_xf"
    value {
      float_list {
        value: 0.18325766921043396
      }
    }
  }
  feature {
    key: "DMC_xf"
    value {
      float_list {
        value: 0.4082949757575989
      }
    }
  }
  feature {
    key: "FFMC_xf"
    value {
      float_list {
        value: 0.7053141593933105
      }
    }
  }
  feature {
    key: "ISI_xf"
    value {
      float_list {
        value: 1.7617137432098389
      }
    }
  }
  feature {
    key: "RH_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "X_xf"
    value {
      float_list {
        value: 0.16529320180416107
      }
    }
  }
  feature {
    key: "Y_xf"
    value {
      float_list {
        value: -0.23549708724021912
      }
    }
  }
  feature {
    key: "area_xf"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "day_xf"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: 

2022-07-30 15:05:53.196204: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled


## Presenting as a dataframe

In [22]:
sample_df = pd.DataFrame.from_records(example_records, columns=int_features+float_features)
sample_df.head()

RH_xf day_xf month_xf                  DC_xf                 DMC_xf  \
0   [1]    [5]      [0]  [0.18325766921043396]   [0.4082949757575989]   
1   [1]    [4]      [0]  [0.06929662823677063]  [0.25802576541900635]   
2   [1]    [1]      [0]   [0.6091344356536865]   [1.7497596740722656]   
3   [0]    [6]      [0]   [0.5386473536491394]   [1.6041864156723022]   
4   [1]    [0]      [8]   [-2.265638589859009]  [-1.5749441385269165]   

                 FFMC_xf                 ISI_xf                 rain_xf  \
0   [0.7053141593933105]   [1.7617137432098389]  [-0.08982300013303757]   
1   [0.7210833430290222]  [0.25552359223365784]  [-0.08982300013303757]   
2   [0.6580066084861755]   [0.5856475234031677]  [-0.08982300013303757]   
3   [0.7210833430290222]    [1.823611855506897]  [-0.08982300013303757]   
4  [0.05876835808157921]   [0.6475457549095154]  [-0.08982300013303757]   

                 temp_xf                 wind_xf                   X_xf  \
0   [0.8269943594932556]      [1.35548734664917]  [0.16529320180416107]   
1  [0.22984977066516876]   [-0.9850295186042786]   [1.4553557634353638]   
2   [0.6864895224571228]  [-0.47125765681266785]   [1.0253349542617798]   
3    [2.003720283508301]   [0.32794326543807983]  [-1.1247694492340088]   
4   [-0.999565839767456]      [3.12514591217041]    [0.595314085483551]   

                     Y_xf               area_xf  
0  [-0.23549708724021912]                 [2.0]  
1    [1.3173850774765015]   [2.549999952316284]  
2  [-0.23549708724021912]                 [0.0]  
3  [-0.23549708724021912]                 [0.0]  
4    [-1.011938214302063]  [61.130001068115234]

# Defining the Trainer

Next we define the trainer. As the name explains, the Trainer takes the features generated by the Transform step (along with other metadata), train a model and save it to the disk. These models have a special element called a signature. The signature is very important as it tells the model how to serve requests coming through an API.

In [23]:
%%writefile forest_fires_trainer.py

# Section 15.2

# Classif vs regression signature defs
# https://www.tensorflow.org/tfx/serving/signature_defs

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import forest_fires_constants


_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


# Code listing 15.10
def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


# Code listing 15.8
def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


# Code listing 15.6
def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key)
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity(
              key, num_buckets=len(boundaries)+1
          )
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns += [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity( 
              key,
              num_buckets=num_buckets,
              default_value=0
          )
      ) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES
      )      
  ]

  model = _dnn_regressor(
      columns=real_valued_columns+categorical_columns,
      dnn_hidden_units=[128, 64])
  return model


# Code listing 15.7
def _dnn_regressor(columns, dnn_hidden_units):
  """Build a simple keras regression model.

  Args:
    columns: Feature columns
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Keras model
  """  

  # Keras needs the feature definitions at compile time.  
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })  
  
  output = tf.keras.layers.DenseFeatures(columns)(input_layers)
  for numnodes in dnn_hidden_units:
    output = tf.keras.layers.Dense(numnodes, activation='tanh')(output)
  
  output = tf.keras.layers.Dense(1)(output)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='mean_squared_error',
      optimizer=tf.keras.optimizers.Adam(lr=0.001)
  )
  model.summary(print_fn=absl.logging.info)

  return model


# Code listing 15.9
# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  absl.logging.info("="*50)
  absl.logging.info("Printing the tfx.components.FnArgs object")  
  absl.logging.info(fn_args)
  absl.logging.info("="*50)

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 40)

  model = _build_keras_model()
  
  os.makedirs(os.path.join(fn_args.model_run_dir,'model_performance'), exist_ok=True)

  csv_callback = tf.keras.callbacks.CSVLogger(
    os.path.join(fn_args.model_run_dir,'model_performance', 'performance.csv'), append=False
  )

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      epochs=10,
      callbacks=[csv_callback]
  )

  # https://www.tensorflow.org/tfx/serving/signature_defs
  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(
              model, tf_transform_output
          ).get_concrete_function(
              tf.TensorSpec(
                  shape=[None],
                  dtype=tf.string,
                  name='examples'
              )
          ),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Writing forest_fires_trainer.py


## Understanding `tf.feature_column`

Below, we show the transformations done through `tf.feature_column` objects using toy examples.

In [24]:
# https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column

a = tf.feature_column.numeric_column("a")
x = tf.keras.layers.DenseFeatures(a)({'a': [0.5, 0.6]})
print(x)

b = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity('b', num_buckets=10))
y = tf.keras.layers.DenseFeatures(b)({'b': [5, 2]})
print(y)



tf.Tensor(
[[0.5]
 [0.6]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]], shape=(2, 10), dtype=float32)


## Running the trainer

In [25]:
# Section 15.2

from tfx.components import Trainer
from tfx.proto import trainer_pb2
import tensorflow.keras.backend as K

K.clear_session()

n_dataset_size = df.shape[0]
batch_size = 40

n_train_steps_mod = 2*n_dataset_size % (3*batch_size)
n_train_steps = int(2*n_dataset_size/(3*batch_size))
if n_train_steps_mod != 0:
    n_train_steps += 1

n_eval_steps_mod = n_dataset_size % (3*batch_size)
n_eval_steps = int(n_dataset_size/(3*batch_size))
if n_eval_steps != 0:
    n_eval_steps += 1
    
trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=n_train_steps),
    eval_args=trainer_pb2.EvalArgs(num_steps=n_eval_steps))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/bin/python', '/tmp/tmpl5i9bmv8/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpq2iypmog', '--dist-dir', '/tmp/tmpxs1a_27v']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying forest_fires_constants.py -> build/lib
copying forest_fires_transform.py -> build/lib
copying forest_fires_trainer.py -> build/lib
installing to /tmp/tmpq2iypmog
running install
running install_lib
copying build/lib/forest_fires_constants.py -> /tmp/tmpq2iypmog
copying build/lib/forest_fires_transform.py -> /tmp/tmpq2iypmog
copying build/lib/forest_fires_trainer.py -> /tmp/tmpq2iypmog
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt


/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:

writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpq2iypmog/tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3.8.egg-info
running install_scripts
creating /tmp/tmpq2iypmog/tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782.dist-info/WHEEL
creating '/tmp/tmpxs1a_27v/tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl' and adding '/tmp/tmpq2iypmog' to it
adding 'forest_fires_constants.py'
adding 'forest_fires_trainer.py'
adding 'forest_fires_transform.py'
adding 'tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782.dist-info/METADATA'
adding 'tfx

INFO:absl:Successfully installed '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl'.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving. ModelRun written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/5
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 11
        type_id: 23
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 12
        type_id: 24
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Evaluate the trained model

Here we are going to manually load the model and make sure it can process a given set of input data.

### Load the model from disk

In [26]:
model = load_model(os.path.join(trainer.outputs["model"]._artifacts[0].uri, "Format-Serving"))
model.summary()


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf675d8940> and <keras.engine.input_layer.InputLayer object at 0x7fdf6616e880>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf675d8940> and <keras.engine.input_layer.InputLayer object at 0x7fdf6616e880>).


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 DC_xf (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 DMC_xf (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 FFMC_xf (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 ISI_xf (InputLayer)            [(None,)]            0           []                               
                                                                                              

### Generate samples using the test file input

Here we generate a small sample of data to test the model. The data needs to be in a very specific format. The specifics are discussed in Chapter 15 of the book.

In [27]:
# Section 15.2

#https://www.tensorflow.org/tfx/transform/api_docs/python/tft/TFTransformOutput
    
import tensorflow_transform as tft
tf_transform_output = tft.TFTransformOutput(transform.outputs["transform_graph"]._artifacts[0].uri)

print("="*10, " Feature Specifications ", "="*10)
print(tf_transform_output.raw_feature_spec()) 
print("="*50, "\n")

# Code listing 15.11

# Load data
test_df = pd.read_csv(os.path.join(test_path, 'forestfires.csv'), index_col=None, header=0)

# Sample contains a dictionary of feature_name -> ragged tensor
sample = {}
for col, val in test_df.to_dict(orient='list').items():
    
    if col=='area':
        continue
    
    # Transformation came from https://github.com/tensorflow/transform/blob/d401253873a2079c245049572e8488347adb3cd4/examples/census_example_v2.py#L92
    if isinstance(val[0], int):
        sample[col] = tf.RaggedTensor.from_tensor(tf.cast(tf.expand_dims(val, -1), dtype='int64')).to_sparse()
        
    else:
        sample[col] = tf.RaggedTensor.from_tensor(tf.expand_dims(val, -1)).to_sparse()

print("="*10, " Sample ", "="*10)
print(sample)
print("="*50, "\n")

==========  Feature Specifications  ==========
{'DC': VarLenFeature(dtype=tf.float32), 'DMC': VarLenFeature(dtype=tf.float32), 'FFMC': VarLenFeature(dtype=tf.float32), 'ISI': VarLenFeature(dtype=tf.float32), 'RH': VarLenFeature(dtype=tf.int64), 'X': VarLenFeature(dtype=tf.int64), 'Y': VarLenFeature(dtype=tf.int64), 'area': VarLenFeature(dtype=tf.float32), 'day': VarLenFeature(dtype=tf.string), 'month': VarLenFeature(dtype=tf.string), 'rain': VarLenFeature(dtype=tf.float32), 'temp': VarLenFeature(dtype=tf.float32), 'wind': VarLenFeature(dtype=tf.float32)}

==========  Sample  ==========
{'X': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fe0c84014c0>, 'Y': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fdf94fbd340>, 'month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fdf67e86100>, 'day': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fdf67e862b0>, 'FFMC': <tensorflow.python.framework.sparse_tens

### Predict with the loaded model

In [28]:
transformed_features = tf_transform_output.transform_features_layer()(sample)

predictions = model.predict(transformed_features)

print(predictions)

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


[[11.528167  ]
 [11.064485  ]
 [10.620287  ]
 [ 8.902476  ]
 [-3.115636  ]
 [12.056705  ]
 [ 7.1407304 ]
 [ 7.6846137 ]
 [ 8.975954  ]
 [ 7.5344057 ]
 [11.219084  ]
 [ 8.800179  ]
 [ 9.722062  ]
 [ 6.6977777 ]
 [12.963372  ]
 [12.675009  ]
 [ 7.243203  ]
 [ 5.078743  ]
 [10.522747  ]
 [ 6.8620505 ]
 [11.720803  ]
 [13.020282  ]
 [ 0.26623622]
 [13.139455  ]
 [ 7.975074  ]
 [10.326383  ]]


# Treating anomalies in the data

Anomalies can a big problem if left unaccounted for. It is important to know how anomalies can be handled in our pipeline. To detect anomalies, we are going to modify the schema we generated earlier.

## Show the existing schema

In [29]:
# Section 15.3

context.show(schema_gen.outputs['schema'])

Type  Presence Valency   Domain
Feature name                                   
'DC'           FLOAT  required  single        -
'DMC'          FLOAT  required  single        -
'FFMC'         FLOAT  required  single        -
'ISI'          FLOAT  required  single        -
'RH'             INT  required  single        -
'X'              INT  required  single        -
'Y'              INT  required  single        -
'area'         FLOAT  required  single        -
'day'         STRING  required  single    'day'
'month'       STRING  required  single  'month'
'rain'         FLOAT  required  single        -
'temp'         FLOAT  required  single        -
'wind'         FLOAT  required  single        -

Values
Domain                                                                                     
'day'                                       'fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed'
'month'  'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'oct', 'sep', 'nov'

## Adapting the schema to identify anomalies

To detect anomalies, we have to introduce constraints to the schema. This is done by providing values to certain attributes found for each feature in the schema. These attributes are defined at `https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto`

In [30]:
# Section 15.3

# Use this for detecting what anomly conditions are detected
#https://www.tensorflow.org/tfx/data_validation/anomalies

# Code listing 15.12
schema = tfdv.load_schema_text(os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt'))

y_feature = tfdv.get_feature(schema, 'Y')
isi_feature = tfdv.get_feature(schema, 'ISI')
ffmc_feature = tfdv.get_feature(schema, 'FFMC')
rain_feature = tfdv.get_feature(schema, 'rain')
area_feature = tfdv.get_feature(schema, 'area')

y_feature.int_domain.max = 7
isi_feature.float_domain.max = 30.0
ffmc_feature.float_domain.min = 60.0
rain_feature.float_domain.max = 1.0
area_feature.float_domain.max = 75.0

statistics = tfdv.generate_statistics_from_csv(os.path.join('data', 'csv', 'train', 'forestfires.csv'))
anomalies = tfdv.validate_statistics(statistics = statistics, schema = schema)
#anomalies = tfdv.validate_examples_in_csv(os.path.join('data', 'csv', 'forestfires.csv'))

tfdv.display_anomalies(anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Anomaly short description  \
Feature name                             
'rain'             Out-of-range values   
'FFMC'             Out-of-range values   
'area'             Out-of-range values   
'Y'                Out-of-range values   
'ISI'              Out-of-range values   

                                                      Anomaly long description  
Feature name                                                                    
'rain'             Unexpectedly high value: 6.4>1(upto six significant digits)  
'FFMC'           Unexpectedly low values: 18.7<60(upto six significant digits)  
'area'        Unexpectedly high value: 1090.84>75(upto six significant digits)  
'Y'                                               Unexpectedly large value: 9.  
'ISI'            Unexpectedly high value: 56.1>30(upto six significant digits)

## Overwrite the schema

Once we make the changes, we have to write back the `schema.pbtxt` file using the new schema.

In [31]:
tfdv.write_schema_text(schema, os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt'))

# Validating examples using the ExampleValidator

We have modified our schema to detect anomalies. With that, we are going to place an ExampleValidator in our pipeline, that will ensure that there are no anomalies present in the data.

In [32]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/6/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/6/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 13
        type_id: 21
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## View anomalies

You can see that the Example validator is picking up the anomalies we wanted to detect.

In [33]:
# Section 15.3

context.show(example_validator.outputs['anomalies'])

Anomaly short description  \
Feature name                             
'FFMC'             Out-of-range values   
'rain'             Out-of-range values   
'ISI'              Out-of-range values   
'Y'                Out-of-range values   
'area'             Out-of-range values   

                                                      Anomaly long description  
Feature name                                                                    
'FFMC'           Unexpectedly low values: 18.7<60(upto six significant digits)  
'rain'             Unexpectedly high value: 6.4>1(upto six significant digits)  
'ISI'            Unexpectedly high value: 56.1>30(upto six significant digits)  
'Y'                                               Unexpectedly large value: 9.  
'area'        Unexpectedly high value: 1090.84>75(upto six significant digits)

Anomaly short description  \
Feature name                             
'area'             Out-of-range values   
'Y'                Out-of-range values   

                                                     Anomaly long description  
Feature name                                                                   
'area'        Unexpectedly high value: 212.88>75(upto six significant digits)  
'Y'                                              Unexpectedly large value: 9.

## Remove anomalies

Once detected, we will remove the anomalies and write the clean data to the disk.

In [34]:
df = pd.read_csv(os.path.join('data','csv', 'train','forestfires.csv'), header=0)

print("Original shape: {}".format(df.shape))
valid_df = df[(df["FFMC"]>=60)&(df["ISI"]<=30)&(df["Y"]<=7)&(df["area"]<=75)&(df["rain"]<=1)]
print("After anomalies removed shape: {}".format(valid_df.shape))
os.makedirs(os.path.join('data','csv_valid', 'train'),exist_ok=True)
valid_df.to_csv(os.path.join('data','csv_valid','train','forestfires.csv'), header=True, index=False)

Original shape: (491, 13)
After anomalies removed shape: (465, 13)


## View statistics of data side by side (before and after removing anomalies)

In [35]:
# Section 15.3

dataset1_stats = tfdv.generate_statistics_from_csv(
  data_location=os.path.join('data', 'csv', 'train', 'forestfires.csv'), delimiter=',')
dataset2_stats = tfdv.generate_statistics_from_csv(
  data_location=os.path.join('data', 'csv_valid', 'train', 'forestfires.csv'), delimiter=',')

tfdv.visualize_statistics(
  lhs_statistics=dataset1_stats, lhs_name='Original data',
  rhs_statistics=dataset2_stats, rhs_name='Data without anomalies'
)


# Rerun the steps after removing anomalies

Here we rerun the following TFX components.

1. `CsvExampleGen` - Load the newly written clean data
2. `StatisticsGen` - Generate statistics for the new data
3. `ExampleValidator` - Ensure there are no anomalies
4. `Transform` - Transform the columns to features as defined earlier


In [36]:
# Section 15.3

# Code listing 15.13

from tfx.components import Transform

#_pipeline_root_clean = os.path.join(os.getcwd(), 'pipeline', 'examples', 'forest_fires_pipeline_clean')
#context = InteractiveContext(pipeline_name = "forest_fires_clean", pipeline_root=_pipeline_root_clean)

# Define the example generator
example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv_valid', 'train'))
# Run the example generator
context.run(example_gen)

# Define the statistic generator
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)
# Run the statistic  generator
context.run(statistics_gen)

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'))

context.run(transform)



INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating examples.
INFO:absl:Processing input csv data data/csv_valid/train/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/ex

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying forest_fires_constants.py -> build/lib
copying forest_fires_transform.py -> build/lib
copying forest_fires_trainer.py -> build/lib
installing to /tmp/tmp1ix0wi5g
running install
running install_lib
copying build/lib/forest_fires_constants.py -> /tmp/tmp1ix0wi5g
copying build/lib/forest_fires_transform.py -> /tmp/tmp1ix0wi5g
copying build/lib/forest_fires_trainer.py -> /tmp/tmp1ix0wi5g
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt


/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Tra

writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp1ix0wi5g/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3.8.egg-info
running install_scripts
creating /tmp/tmp1ix0wi5g/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782.dist-info/WHEEL
creating '/tmp/tmpib4qrnfs/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl' and adding '/tmp/tmp1ix0wi5g' to it
adding 'forest_fires_constants.py'
adding 'forest_fires_trainer.py'
adding 'forest_fires_transform.py'
adding 'tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782.dist-info/MET

INFO:absl:Successfully installed '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+cba4cab94c92ffc629ac9b0a3b7ecf77bb34e3a96d4553d4cdd81edad259e782-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/bin

...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 18
        type_id: 20
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/11"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 19
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/11"
        properties {
          key: "split_names"
          value {
            string_value: "[\"eval\", \"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 20
        type_id: 26
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/updated_analyzer_cache/11"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 21
        type_id: 18
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/pre_transform_schema/11"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
   

## Show statistics for the new data

In [37]:
context.show(statistics_gen.outputs['statistics'])

## Show example validation results for the new data

In [38]:
context.show(example_validator.outputs['anomalies'])

# Train the model (after removing anomalies)

Here we retrain the model with new data. However, it will not replace the existing model. Rather, it will create a new model with a higher execution ID.

In [39]:
%%writefile forest_fires_trainer_v2.py

# Section 15.3

# Classif vs regression signature defs
# https://www.tensorflow.org/tfx/serving/signature_defs

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import forest_fires_constants


_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, targets) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of targets.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key)
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity(
              key, num_buckets=len(boundaries)+1
          )
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns += [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity( 
              key,
              num_buckets=num_buckets,
              default_value=0
          )
      ) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES
      )      
  ]

  model = _dnn_regressor(
      columns=real_valued_columns+categorical_columns,
      dnn_hidden_units=[128, 64])
  return model


def _dnn_regressor(columns, dnn_hidden_units):
  """ Build a simple keras regressor model. """
  
  # Keras needs the feature definitions at compile time.  
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })  

  output = tf.keras.layers.DenseFeatures(columns)(input_layers)
  for numnodes in dnn_hidden_units:
    output = tf.keras.layers.Dense(numnodes, activation='tanh')(output)
  
  output = tf.keras.layers.Dense(1)(output)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='mean_squared_error',
      optimizer=tf.keras.optimizers.Adam(lr=0.0001)
  )
  model.summary(print_fn=absl.logging.info)

  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  absl.logging.info("="*50)
  absl.logging.info("Printing the tfx.components.FnArgs object")  
  absl.logging.info(fn_args)
  absl.logging.info("="*50)

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, fn_args.custom_config["batch_size"])
    
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, fn_args.custom_config["batch_size"])
  
  
  model = _build_keras_model()
  
  os.makedirs(os.path.join(fn_args.model_run_dir,'model_performance'), exist_ok=True)

  csv_callback = tf.keras.callbacks.CSVLogger(
    os.path.join(fn_args.model_run_dir,'model_performance', 'performance.csv'), append=False
  )

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      epochs=fn_args.custom_config["epochs"],
      callbacks=[csv_callback]
  )

  # https://www.tensorflow.org/tfx/serving/signature_defs
  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(
              model, tf_transform_output
          ).get_concrete_function(
              tf.TensorSpec(
                  shape=[None],
                  dtype=tf.string,
                  name='examples'
              )
          ),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Writing forest_fires_trainer_v2.py


In [40]:
# Section 15.3

from tfx.components import Trainer
from tfx.proto import trainer_pb2

n_valid_dataset_size = valid_df.shape[0]

batch_size = 40

n_train_steps_mod = 2*n_valid_dataset_size % (3*batch_size)
n_train_steps = int(2*n_valid_dataset_size/(3*batch_size))
if n_train_steps_mod != 0:
    n_train_steps += 1

n_eval_steps_mod = n_valid_dataset_size % (3*batch_size)
n_eval_steps = int(n_valid_dataset_size/(3*batch_size))
if n_eval_steps != 0:
    n_eval_steps += 1


trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer_v2.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=n_train_steps),
    eval_args=trainer_pb2.EvalArgs(num_steps=n_eval_steps),
    custom_config={'batch_size': batch_size, 'epochs': 10}
)

context.run(trainer)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/forest_fires_trainer_v2.py' (including modules: ['forest_fires_constants', 'forest_fires_trainer_v2', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/bin/python', '/tmp/tmp_gs9dtt2/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpnyplkyq1', '--dist-dir', '/tmp/tmpxraunoh4']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying forest_fires_constants.py -> build/lib
copying forest_fires_trainer_v2.py -> build/lib
copying forest_fires_transform.py -> build/lib
copying forest_fires_trainer.py -> build/lib
installing to /tmp/tmpnyplkyq1
running install
running install_lib
copying build/lib/forest_fires_constants.py -> /tmp/tmpnyplkyq1
copying build/lib/forest_fires_trainer_v2.py -> /tmp/tmpnyplkyq1
copying build/lib/forest_fires_transform.py -> /tmp/tmpnyplkyq1
copying build/lib/forest_fires_trainer.py -> /tmp/tmpnyplkyq1
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt


/home/thushv89/anaconda3/envs/manning.tf2.9.tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d-py3-none-any.whl'; target user module is 'forest_fires_trainer_v2'.
INFO:absl:Full user module path is 'forest_fires_trainer_v2@/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer

writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpnyplkyq1/tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d-py3.8.egg-info
running install_scripts
creating /tmp/tmpnyplkyq1/tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d.dist-info/WHEEL
creating '/tmp/tmpxraunoh4/tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d-py3-none-any.whl' and adding '/tmp/tmpnyplkyq1' to it
adding 'forest_fires_constants.py'
adding 'forest_fires_trainer.py'
adding 'forest_fires_trainer_v2.py'
adding 'forest_fires_transform.py'
adding 'tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e2

INFO:absl:Successfully installed '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+cf07c65688ecc3ff3d4674ff4ff4cafefbaa29898f88622cca144df445e28b4d-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:==================================================
INFO:absl:Printing the tfx.components.FnArgs object
INFO:absl:FnArgs(working_dir=None, train_files=['/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/11/Split-train/*'], eval_files=['/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/11/Split-eval/*'], train_steps=8, eval_steps=4, schema_path='/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/9/schema.pbtxt', schema_file='/home/thu

...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12/Format-Serving. ModelRun written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/12
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 12
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 26
        type_id: 23
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 27
        type_id: 24
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/12"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Printing the location of the output file

In [41]:
trainer.outputs['model']._artifacts[0].uri

'/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12'

# Validating the Infrastructure of the model

After the model is trained, we are going to look at the model serving infrastructure. We will use a Docker container to serve our model. There is a TFX component known as InfraValidator that will,

* Stand up the container
* Load the model into the container
* Send dummy requests
* Verify the model responds

In [43]:
# Section 15.5

# Code listing 15.14
from tfx.components import InfraValidator
from tfx.proto import infra_validator_pb2
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    # This is the source for the data that will be used to build a request.
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        # Depending on what kind of model server you're using, RequestSpec
        # should specify the compatible one.
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(tags=['2.6.3-gpu']),
        local_docker=infra_validator_pb2.LocalDockerConfig(),
    ),
    request_spec=infra_validator_pb2.RequestSpec(
        # InfraValidator will look at how "classification" signature is defined
        # in the model, and automatically convert some samples from `examples`
        # artifact to prediction RPC requests.
        tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(
            signature_names=['serving_default']
        ),
        num_examples=5  # How many requests to make.
    )
)

context.run(infra_validator)

INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for InfraValidator
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl:Creating temp directory at /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/.temp/14/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:2.8.2-gpu).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:2.8.2-gpu', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'Source': '/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/.temp/14/infra-validation-model/16591

ExecutionResult(
    component_id: InfraValidator
    execution_id: 14
    outputs:
        blessing: Channel(
            type_name: InfraBlessing
            artifacts: [Artifact(artifact: id: 29
        type_id: 28
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/InfraValidator/blessing/14"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "InfraValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "InfraBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

# Resolving the model

The resolver gives a baseline model to the downstream Evaluator component. The Evaluator uses the baseline model to asses how good the newly trained model is. 

In [44]:
from tfx import v1 as tfx

model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

context.run(model_resolver)



INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 15
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

# Validating/Evaluating the model

Here we define some criteria the TFX pipeline can use to sign off a model. To sign off a model we can define certain criteria like,

* Have a loss value below a specified threshold
* Be at least x% better than the previous model
* etc.


In [45]:
# Section 15.5

import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key='area')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='MeanSquaredError',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          upper_bound={'value': 200.0}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.LOWER_IS_BETTER,
                          absolute={'value': 1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column month.
        tfma.SlicingSpec(feature_keys=['month'])
    ])


In [46]:
from tfx.components import Evaluator

# You might see an error such as,
# tensorflow.python.framework.errors_impl.InvalidArgumentError: 'func' argument to TF_GraphCopyFunction cannot be null
# This is a bug in TensorFlow (more info: https://github.com/tensorflow/tensorflow/issues/44403)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)


INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 200.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, '


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf6442e250> and <keras.engine.input_layer.InputLayer object at 0x7fdf6444b1f0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf6442e250> and <keras.engine.input_layer.InputLayer object at 0x7fdf6444b1f0>).
INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 200.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf52b301f0> and <keras.engine.input_layer.InputLayer object at 0x7fdf52ac2c40>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf52b301f0> and <keras.engine.input_layer.InputLayer object at 0x7fdf52ac2c40>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf01ac35e0> and <keras.engine.input_layer.InputLayer object at 0x7fdf01b44790>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf01ac35e0> and <keras.engine.input_layer.InputLayer object at 0x7fdf01b44790>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf65aa75b0> and <keras.engine.input_layer.InputLayer object at 0x7fdf8c53d760>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf65aa75b0> and <keras.engine.input_layer.InputLayer object at 0x7fdf8c53d760>).
INFO:absl:Evaluation complete. Results written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/16.
INFO:absl:Checking validation results.
INFO:absl:Blessing result False written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/16.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 16
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 30
        type_id: 31
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/16"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 31
        type_id: 32
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/16"
        custom_properties {
          key: "blessed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 26
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Analysing the Evaluator's results more closely

In [47]:
# Section 15.5

import tensorflow_model_analysis as tfma

validation_path = os.path.join(
    evaluator.outputs['evaluation']._artifacts[0].uri, "validations"
)
validation_res = tfma.load_validation_result(validation_path)

print('='*20, " Output stored in validations file ", '='*20)
print(validation_res)
print("="*75)

metrics_path = os.path.join(
    evaluator.outputs['evaluation']._artifacts[0].uri, "metrics"
)
metrics_res = tfma.load_metrics(metrics_path)

print('='*20, " Output stored in metrics file ", '='*20)
for r in metrics_res:
    print(r)
    print('-'*75)
print("="*75)

====================  Output stored in validations file  ====================
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "month"
      bytes_value: "sep"
    }
  }
  failures {
    metric_key {
      name: "mean_squared_error"
      example_weighted {
      }
    }
    metric_threshold {
      value_threshold {
        upper_bound {
          value: 200.0
        }
      }
    }
    metric_value {
      double_value {
        value: 269.11712646484375
      }
    }
  }
}
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 12
  }
}

====================  Output stored in metrics file  ====================
slice_key {
}
metric_keys_and_values {
  key {
    name: "loss"
  }
  value {
    double_value {
      value: 160.19691467285156
    }
  }
}
metric_keys_and_values {
  key {
    name: "mean_squared_error"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 160.19691467285156
    }
  }


## Validating the model with updated metric thresholds

The above model will not meet the criteria by looking at the logs produced by the Evaluator. Here we update the criteria slightly (setting the `value_threshold` to 300) to make sure the model passes.

In [48]:
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='area')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='MeanSquaredError',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          upper_bound={'value': 300.0}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.LOWER_IS_BETTER,
                          absolute={'value': 1e-10})))
            ]
        )
    ],
    slicing_specs=[        
        tfma.SlicingSpec(),        
        tfma.SlicingSpec(feature_keys=['month'])
    ])


evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config,
)

context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 300.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, '


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf015bb220> and <keras.engine.input_layer.InputLayer object at 0x7fdf0156c8b0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf015bb220> and <keras.engine.input_layer.InputLayer object at 0x7fdf0156c8b0>).
INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 300.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf21b630a0> and <keras.engine.input_layer.InputLayer object at 0x7fdf21aff700>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf21b630a0> and <keras.engine.input_layer.InputLayer object at 0x7fdf21aff700>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf082a6b20> and <keras.engine.input_layer.InputLayer object at 0x7fdf082d3850>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdf082a6b20> and <keras.engine.input_layer.InputLayer object at 0x7fdf082d3850>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdef07bdd00> and <keras.engine.input_layer.InputLayer object at 0x7fdef07a0940>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fdef07bdd00> and <keras.engine.input_layer.InputLayer object at 0x7fdef07a0940>).
INFO:absl:Evaluation complete. Results written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/17.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/17.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 17
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 32
        type_id: 31
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/17"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 33
        type_id: 32
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/17"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/12"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 26
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Viewing the validation results (successful)

In [49]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))


validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 12
  }
}



# Pushing the model to the deployment environment

Everything is ready. Once the model passes evaluation criteria. We push the final model to a given destination (i.e. a local folder in this case).

In [50]:
# Section 15.5

from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
  model=trainer.outputs['model'],
  model_blessing=evaluator.outputs['blessing'],
  infra_blessing=infra_validator.outputs['blessing'],  
  push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=os.path.join('forestfires-model-pushed'))
  )
)
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1659157653
INFO:absl:Model written to serving path forestfires-model-pushed/1659157653.
INFO:absl:Model pushed to /home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Pusher/pushed_model/18.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 18
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 34
        type_id: 34
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch15-TFX-for-MLOps-in-TF2/tfx/pipeline/examples/forest_fires_pipeline/Pusher/pushed_model/18"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "forestfires-model-pushed/1659157653"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1659157653"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.6.0"
          }
        }
        state: LIVE
        , artifact_type: id: 34
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Manually pushing some of the remaining artifacts

`tfx` in its current state is quite buggy. It only copys the `saved_model.pb` and ignores the other important files such as the `Assets` folder. Here we copy them manually so we can generate predictions during serving.

In [97]:
import shutil

def push_manually(tfx_pusher):
    
    src_dir = tfx_pusher.outputs["pushed_model"]._artifacts[0].uri
    dest_dir = pusher.outputs['pushed_model']._artifacts[0]._artifact.custom_properties["pushed_destination"].string_value
    
    shutil.copytree(src_dir, dest_dir, dirs_exist_ok=True)
    
push_manually(pusher)

# Using the served model

To use the served model we have to start a Docker container (based on the TensorFlow serving Docker Image) that has the model and can serve HTTP requests. 

## Running the TensorFlow serving server

1. Open up a terminal and `cd` into the `Ch15` directory
2. Run the model serving service with `./run_server.sh`

## List metadata of the API

You can look at the signatures of the model by pinging the following endpoint.

In [51]:
import json 

json_response = requests.get(
    'http://localhost:8501/v1/models/forest_fires_model/metadata'
)
predictions = json.loads(json_response.text)

print(predictions)

{'model_spec': {'name': 'forest_fires_model', 'signature_name': '', 'version': '1659157653'}, 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'examples': {'dtype': 'DT_STRING', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_examples:0'}}, 'outputs': {'output_0': {'dtype': 'DT_FLOAT', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '1', 'name': ''}], 'unknown_rank': False}, 'name': 'StatefulPartitionedCall_10:0'}}, 'method_name': 'tensorflow/serving/predict'}, '__saved_model_init_op': {'inputs': {}, 'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID', 'tensor_shape': {'dim': [], 'unknown_rank': True}, 'name': 'NoOp'}}, 'method_name': ''}}}}}


## Predict with the API

In [52]:
# Section 15.5

import json
import requests
import base64

req_body = {
  "signature_name": "serving_default",

  "instances": 
    [
            str(base64.b64encode(
                b"{\"X\": 7,\"Y\": 4,\"month\":\"feb\",\"day\":\"fri\",\"FFMC\":20,\"DMC\":30,\"DC\":40,\"ISI\":2,\"temp\":10,\"RH\":50,\"wind\":50,\"rain\":0}]")
               )
    ]
    
}

data = json.dumps(req_body)

json_response = requests.post(
    'http://localhost:8501/v1/models/forest_fires_model:predict', 
    data=data, 
    headers={"content-type": "application/json"}
)
predictions = json.loads(json_response.text)

print(predictions)


{'predictions': [[-0.544334829]]}


In [53]:
import json
import requests
import base64

req_body = {
  "signature_name": "serving_default",

  "instances": 
    [
        str(base64.b64encode(
            b"{\"X\": 9,\"Y\": 6,\"month\":\"aug\",\"day\":\"fri\",\"FFMC\":91,\"DMC\":248,\"DC\":553,\"ISI\":6,\"temp\":20.5,\"RH\":58,\"wind\":3,\"rain\":0}]")
        ),
        str(base64.b64encode(
            b"{\"X\": 9,\"Y\": 6,\"month\":\"aug\",\"day\":\"fri\",\"FFMC\":91,\"DMC\":248,\"DC\":553,\"ISI\":6,\"temp\":20.5,\"RH\":58,\"wind\":3,\"rain\":0}]")
        ),
        
    ]
    
}

data = json.dumps(req_body)

json_response = requests.post(
    'http://localhost:8501/v1/models/forest_fires_model:predict', 
    data=data, 
    headers={"content-type": "application/json"}
)
predictions = json.loads(json_response.text)

print(predictions)


{'predictions': [[-0.544335246], [-0.544335246]]}
